In [17]:
import numpy as np
import pandas as pd
import ast
from operator import itemgetter

In [27]:
## merging two datasets
data=pd.read_csv("tmdb_5000_movies.csv")
data1=pd.read_csv("tmdb-movies.csv")

data=data.drop(['homepage','overview','production_countries','spoken_languages','status','tagline'],axis=1)

data1=data1.iloc[:,[0,5,8,18]]

data2=pd.merge(data,data1,on='id')

data2=data2.drop(["original_title_x","original_title_y",'release_date'],axis=1)

df = data2.replace('[]',None)
df = df.dropna()

df = df.drop(["budget"], axis = 1)

In [21]:
df.head(5)

,genres,id,keywords,original_language,popularity,production_companies,release_date,revenue,runtime,title,vote_average,vote_count,director
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,Avatar,7.2,11800,James Cameron
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,Gore Verbinski
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,880674609,148.0,Spectre,6.3,4466,Sam Mendes
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,1084939099,165.0,The Dark Knight Rises,7.6,9106,Christopher Nolan
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,284139100,132.0,John Carter,6.1,2124,Andrew Stanton


In [4]:
##dummification for genres
genres = set() #set of unique genres
for i in range(len(df)):
    for genre_dict in ast.literal_eval(df.iloc[i,0]):
        genres.add(genre_dict['name'])
genres    

#adding 20 new columns for unique genres
for genre in genres:
    df['genre_' + genre] = 0

#dummification
for i in range(len(df)):
    for genre_dic in ast.literal_eval(df.iloc[i,0]):
        genre = genre_dic['name']
        df.loc[i, "genre_" + genre] = 1 #value of dummy variable is 1 if the genre is present

df = df.drop(["genres"], axis = 1) # dropping 'genres' column after dummification

In [5]:
##############################################################################################

In [6]:
##dummification for production companies
prodc = {} #set of unique genres
for i in range(len(df)):
    for prodc_dict in ast.literal_eval(df.iloc[i,4]):
        try:
            prodc[prodc_dict['id']] += 1
        except:
            prodc[prodc_dict['id']] = 1 

sorted_prodc = list(sorted(prodc.items(), key=lambda x: x[1], reverse = True))[:9]

famous_prodc = []
for i in range(8):
    famous_prodc.insert(i, sorted_prodc[i][0])

#adding 10 columns for production houses
for id in famous_prodc:
    df['prodc_' + str(id)] = 0
df['prodc_l40'] = 0 #production company with less than 40 movies
df['prodc_g40'] = 0 ##production company with greater than 40 movies

for i in range(len(df)):
    prodc_dic = ast.literal_eval(df.loc[i,"production_companies"])
    companies = list(map(itemgetter('id'), prodc_dic))  #list of ids of production compnaies for i-th movie
    for company in companies:
        if company in famous_prodc:
            df.loc[i, 'prodc_'+str(company)] = 1
        elif prodc[company] < 40:
            df.loc[i, 'prodc_l40'] = 1
        else:
            df.loc[i, 'prodc_g40'] = 1

df = df.drop('production_companies', axis = 1) #dropping the original production company columns

In [7]:
for i in range(len(df)):
    if df.loc[i, "original_language"] == 'en':
        df.loc[i, "original_language"] = 1
    else:
        df.loc[i, "original_language"] = 0

In [8]:
############################################################################################

In [9]:
keywords = {} #set of unique genres
for i in range(len(df)):
    for kw_dict in ast.literal_eval(df.iloc[i,1]):
        try:
            keywords[kw_dict['name']] += 1
        except:
            keywords[kw_dict['name']] = 1 

fam_kw = [x[0] for x in list(sorted(keywords.items(), key=lambda x: x[1], reverse = True))[1:12]]
fam_kw.remove('aftercreditsstinger')

#adding 10 new columns for famous keywords
for kw in fam_kw:
    df['kw_' + kw] = 0

#dummification
for i in range(len(df)):
    kw_dic = ast.literal_eval(df.loc[i,"keywords"])
    kws = list(map(itemgetter('name'),kw_dic ))  #list of names of keywords for i-th movie
    for kw in kws:
        if kw in fam_kw:
            df.loc[i, 'kw_'+str(kw)] = 1
        else:
            pass

df = df.drop('keywords', axis = 1) #dropping the original keywords columns

In [10]:
###############################################################################################

In [12]:
#date to year
for i in range(len(df)):
    date = df.iloc[i, 3]
    year = int(date.split("-")[0])
    df.iloc[i, 3] = year
    
#converting release year into binary variable
for i in range(len(df)):
    if int(df.iloc[i,3])>=2000:
        df.iloc[i,3]=1
    else:
        df.iloc[i,3]=0

In [13]:
#directors
dict1={}
dict2={}
for i in range(len(df)):
    if df.iloc[i,9] not in dict1.keys():
        dict1[df.iloc[i,9]]=1
    else:
        dict1[df.iloc[i,9]]+=1

    if df.iloc[i,9] not in dict2.keys():
        dict2[df.iloc[i,9]]=df.iloc[i,7]
    else:
        dict2[df.iloc[i,9]]+=df.iloc[i,7]     
    

dict={}
lst_other2=[]
dict['others']=0
for i in dict1.keys():
    if dict1[i]<5:
        lst_other2.append(i)
        dict["others"]+=dict1[i]
    else:
        dict[i]=dict1[i]
        
        

dict3={}
for i in dict.keys():
    if i!='others':
        dict3[i]=dict2[i]/dict1[i]
dict3=sorted(dict3.items(),key=lambda x:x[1],reverse=True)

lst_fam=[]
lst_other=[]
for i in range(len(dict3)):
    if i<=15:
        lst_fam.append(dict3[i][0])
    else:
        lst_other.append(dict3[i][0])
        

    

#dummification of directors column
director_other=[]
director_other2=[]
for i in range(len(df)):
    if df.iloc[i,9] in lst_other:
        director_other.append(1)
    else:
        director_other.append(0)
    if df.iloc[i,9] in lst_other2:
        director_other2.append(1)
    else:
        director_other2.append(0)

for dirct in lst_fam:
    df[dirct] = 0

for i in range(len(df)):
    if df.iloc[i,9] in lst_fam:
        df.loc[i, df.iloc[i,9]] = 1
    else:
        pass

df['other_dirct']=director_other
df['other2_dirct']=director_other2

df=df.drop(['director','revenue'],axis=1)
df_for_budget = pd.read_csv("merged data.csv")
df['budget'] = df_for_budget['budget']

In [14]:
## Casts
df1=pd.read_csv("tmdb-movies.csv")
df1=df1.iloc[:,[0,5,6]]
df2=pd.merge(df,df1,on='id')
df2=df2.dropna()
df2=df2.reset_index(drop=True)

cast_dic={}
for i in range(len(df2)):
    lst=[]
    cast=df2.loc[i,'cast']
    lst=cast.split("|")
    for j in range(len(lst)):
        if lst[j] not in cast_dic.keys():
            cast_dic[lst[j]]=1
        else:
            cast_dic[lst[j]]+=1

castrating_dic={}
for i in range(len(df2)):
    lst=[]
    cast=df2.loc[i,'cast']
    lst=cast.split("|")
    for j in range(len(lst)):
        if lst[j] not in castrating_dic.keys():
            castrating_dic[lst[j]]=df2.loc[i,'vote_average']
        else:
            castrating_dic[lst[j]]+=df2.loc[i,'vote_average']
cast_final={}
for i in cast_dic.keys():
    cast_final[i]=[cast_dic[i],(castrating_dic[i]/cast_dic[i])]
cast_final=sorted(cast_final.items(),key=lambda x:x[1],reverse=True)

d = { i:j for i,j in cast_final}
#cast_final=dict((x, y) for x, y in cast_final)  

cast_fam=[]
cast_g10=[]
cast_l10=[]

for i in range(len(d.keys())):
    if i<=17:
        cast_fam.append(list(d.keys())[i])
    else:
        if d[(list(d.keys())[i])][0]>=10:
            cast_g10.append((list(d.keys())[i]))
        else:
            cast_l10.append((list(d.keys())[i]))

            
# Dummicifications for cast columns            
for cast in cast_fam:
    df2[cast]=0
df2['cast_g10']=0
df2['cast_l10']=0
for i in range(len(df2)):
    lst=[]
    cast=df2.loc[i,'cast']
    lst=cast.split("|")
    for j in range(len(lst)):
        if lst[j] in cast_fam:
            df2.loc[i,(lst[j])]=1
        elif lst[j] in cast_g10:
            df2.loc[i,'cast_g10']=1
        else:
            df2.loc[i,'cast_l10']=1

In [15]:
## Re-arranging Numerical and Categorical Columns
df1=df2.drop(['id','genre_Fantasy','prodc_l40','other2_dirct','cast_l10'],axis=1)

title=df1.pop('title')
popularity=df1.pop('popularity')
df1.insert(1, 'popularity', popularity)
run=df1.pop('runtime')
df1.insert(2, 'runtime', run)
v_avg=df1.pop('vote_average')
df1.insert(3, 'vote_average', v_avg)
v_count=df1.pop('vote_count')
df1.insert(4, 'vote_count', v_count)
budget=df1.pop('budget')
df1.insert(5, 'budget', budget)
df1=df1.drop(['original_title', 'cast'],axis=1)

In [17]:
df1.to_csv("Final_Data.csv", index=False)